# OGGM experiment

In [ ]:
# basic
import os 
import numpy as np
import glob as glob
from datetime import datetime

# geospatial
import xarray as xr
import geopandas as gpd

# oggm
from oggm import cfg, utils, workflow, tasks
from oggm.shop import gcm_climate

## Setup and possible options 

In [ ]:
cfg.initialize(logging_level='ERROR')

cfg.PARAMS['use_multiprocessing']  = True
cfg.PARAMS['baseline_climate']     = ''
cfg.PARAMS['prcp_scaling_factor']  = 1
cfg.PARAMS['hydro_month_sh']       = 1
cfg.PARAMS['hydro_month_nh']       = 1
cfg.PARAMS['border']               = 80
cfg.PARAMS['min_mu_star']          = 5
cfg.PARAMS['max_mu_star']          = 600
cfg.PARAMS['geodetic_mb_period']   = '2000-01-01_2020-01-01' 
cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['continue_on_error']    = True
cfg.PARAMS['use_winter_prcp_factor'] = False

# Potential historical scenario
outlines    = ["RGI6", "RGI7"]          # Glacier outlines
outlines    = ["RGI6"]          # Glacier outlines
climate_ds  = ["ERA5", "MSWEP", "PMET", "CR2MET"]  # Climate baseline
climate_ds  = ["MSWEP"]  # Climate baseline
volume_ds   = ["F19", "M22"]            # Reference volume dataset 
volume_ds   = ["M22"]            # Reference volume dataset 

# Potential future scenario
gcm_list  = ["ACCESS-CM2", "BCC-CSM2-MR", "CMCC-ESM2", "FGOALS-f3-L", "GFDL-ESM4", "CMCC-CM2-SR5", "KACE-1-0-G", "MPI-ESM1-2-HR", "MRI-ESM2-0", "MIROC6"] # Climate models
ssp_list  = ["ssp126","ssp245", "ssp370", "ssp585"]     # Future scenarios
bias_correction = ["MVA", "DQM" , "MBC"]           # Bias correction method 

In [ ]:
for rgi in outlines:
    for file_id in climate_ds: 
        for volume in volume_ds:
            
            start = datetime.now()
            
            ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/" + rgi + "_v2.shp")
            ids = ids[ids.area_km2 > 7] # ~ 934 (757) glaciers in RGI6 (RGI7)
            
            cfg.PATHS['working_dir']  = "/home/rooda/OGGM_results/" + rgi + "_" + file_id + "_" + volume +"_run"
            cfg.PATHS['climate_file'] = "/home/rooda/OGGM_results/" + file_id + "_OGGM_1980_2019m.nc"

            if rgi == "RGI6":
                # init directories
                cfg.PARAMS['use_rgi_area'] = True
                cfg.PARAMS['use_intersects'] = True
                cfg.PARAMS['rgi_version'] = 62
                base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L1-L2_files/elev_bands/' # RGI 62
                gdirs = workflow.init_glacier_directories(ids.RGIId, from_prepro_level=2, prepro_border = 80, prepro_base_url = base_url)

            if rgi == "RGI7":
                cfg.PARAMS['use_rgi_area'] = False
                cfg.PARAMS['use_intersects'] = False
                cfg.PARAMS['rgi_version'] = 70
                
                columns_ids = {'glac_id': 'GLIMSId', 'area_km2' : 'Area', 'glac_name': 'Name', 'Zone' : 'Zone', 
                               'vol_F19': 'vol_F19', 'vol_M22':'vol_M22', 'dmdtda_21' :'dmdtda_21'}
                ids = utils.cook_rgidf(ids, o1_region='17', o2_region='02', bgndate= ids.src_date, version = "70",
                              assign_column_values= columns_ids)
            
                gdirs = workflow.init_glacier_directories(ids)
                workflow.execute_entity_task(tasks.define_glacier_region, gdirs, source="NASADEM");

                task_list = [tasks.process_dem, tasks.simple_glacier_masks, tasks.elevation_band_flowline,
                             tasks.fixed_dx_elevation_band_flowline, tasks.compute_downstream_line, tasks.compute_downstream_bedshape]
             
                for task in task_list:
                    workflow.execute_entity_task(task, gdirs);

            # write climate file for each glacier
            workflow.execute_entity_task(tasks.process_custom_climate_data, gdirs);
            
            # calibration using huggonet et al. 2021            
            ref_mb21 = ids.set_index("RGIId").dmdtda_21*1000    
            workflow.execute_entity_task(tasks.mu_star_calibration_from_geodetic_mb, 
                                             [(gdir, {'ref_mb': float(ref_mb21.loc[gdir.rgi_id])}) for gdir in gdirs]);
            #workflow.execute_entity_task(tasks.mu_star_calibration_from_geodetic_mb, gdirs);
            workflow.execute_entity_task(tasks.apparent_mb_from_any_mb, gdirs);
            
            #subset gdirs to avoid infinitive error messages
            df = utils.compile_task_log(gdirs, task_names=["apparent_mb_from_any_mb"])
            df = df[df.apparent_mb_from_any_mb == "SUCCESS"]
            gdirs = [gdir for gdir in gdirs if gdir.rgi_id in df.index.tolist()]

            # inversion by catchment
            for zone in range(1,10):
                ids_subset = ids[ids.Zone == zone]
                gdirs_subset = [gdir for gdir in gdirs if gdir.rgi_id in ids_subset.RGIId.tolist()]

                if volume == "M22": # Millan et al. 2022 
                    workflow.calibrate_inversion_from_consensus(gdirs_subset, volume_m3_reference = ids_subset.vol_M22.sum()*1e9,
                                                                apply_fs_on_mismatch=True, error_on_mismatch=False, 
                                                                filter_inversion_output=True);
                else: # Farinotti et al. 2019
                    workflow.calibrate_inversion_from_consensus(gdirs_subset, volume_m3_reference = ids_subset.vol_F19.sum()*1e9,
                                                                apply_fs_on_mismatch=True, error_on_mismatch=False, 
                                                                filter_inversion_output=True);

            workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs); # ready to use

            workflow.execute_entity_task(tasks.run_with_hydro, 
                             [(gdir, {'ref_dmdtda': float(ref_mb21.loc[gdir.rgi_id])}) for gdir in gdirs], 
                             run_task = tasks.run_dynamic_mu_star_calibration, 
                             store_monthly_hydro=True, 
                             ys=1980,
                             ye=2020,
                             ref_area_from_y0=True,
                             output_filesuffix= "_" + file_id,
                             err_ref_dmdtda = 0.25*1000,
                             ignore_errors=True);

            utils.compile_glacier_statistics(gdirs);  # compile few things
            utils.compile_run_output(gdirs, input_filesuffix= "_" + file_id)

            # future climate tasks
            for gcm in gcm_list:    
                for ssp in ssp_list:
                    for bc in bias_correction:
                        rid = "_{}_{}_{}".format(gcm, ssp, bc)

                        # write future climate file for each glacier
                        workflow.execute_entity_task(gcm_climate.process_cmip_data, gdirs, filesuffix =  rid, 
                                                 fpath_precip = "/home/rooda/OGGM_results/Future_climate_bc/PP_" + file_id + rid + ".nc", 
                                                 fpath_temp = "/home/rooda/OGGM_results/Future_climate_bc/T2M_" + file_id + rid + ".nc", 
                                                 apply_bias_correction=False);

                        # run the glacier using hydro function 
                        workflow.execute_entity_task(tasks.run_with_hydro, gdirs, run_task = tasks.run_from_climate_data,
                                                 climate_filename = 'gcm_data',  # use gcm_data, not climate_historical
                                                 climate_input_filesuffix = rid,  # use the chosen scenario
                                                 init_model_filesuffix = "_" + file_id,  # this is important! Start from 2020 glacier
                                                 ref_geometry_filesuffix = "_" + file_id,  # also use this as area reference
                                                 ref_area_from_y0 = True,  # and keep the same reference area as for the hist simulations
                                                 output_filesuffix = rid,  # recognize the run for later
                                                 store_monthly_hydro = True)  # add monthly diagnostics
                        utils.compile_run_output(gdirs, input_filesuffix=rid)

                    print(rgi, file_id, volume, gcm, ssp, datetime.now()-start)

            # Remove files (model geometry and diagnostics) to save space
            for remove_file in glob.glob(cfg.PATHS['working_dir'] + "/**/model_*.nc",recursive=True):
                os.remove(remove_file)